In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import os
from pathlib import Path
import matplotlib.pyplot as plt

# Initialize MediaPipe face detection and face mesh
mp_face_detection = mp.solutions.face_detection
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

def detect_and_split_face(image_path, output_dir):
    """
    Detect face, find nose tip, and split face into left and right parts
    """
    # Read image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Could not read image: {image_path}")
        return None, None, (None, None), None
    
    # Convert BGR to RGB
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Initialize face mesh
    with mp_face_mesh.FaceMesh(
        static_image_mode=True,
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5
    ) as face_mesh:
        
        # Process the image
        results = face_mesh.process(rgb_image)
        
        if not results.multi_face_landmarks:
            print("No face detected in the image")
            return None, None, (None, None), None
        
        # Get face landmarks
        face_landmarks = results.multi_face_landmarks[0]
        h, w, c = image.shape
        
        # Get nose tip landmark (index 1 in MediaPipe face mesh)
        nose_tip = face_landmarks.landmark[1]
        nose_x = int(nose_tip.x * w)
        nose_y = int(nose_tip.y * h)
        
        # Get face bounding box
        x_coords = [landmark.x * w for landmark in face_landmarks.landmark]
        y_coords = [landmark.y * h for landmark in face_landmarks.landmark]
        
        x_min, x_max = int(min(x_coords)), int(max(x_coords))
        y_min, y_max = int(min(y_coords)), int(max(y_coords))
        
        # Add some padding
        padding = 20
        x_min = max(0, x_min - padding)
        y_min = max(0, y_min - padding)
        x_max = min(w, x_max + padding)
        y_max = min(h, y_max + padding)
        
        # Extract face region
        face_roi = image[y_min:y_max, x_min:x_max]
        
        # Adjust nose tip coordinates relative to face ROI
        nose_x_rel = nose_x - x_min
        nose_y_rel = nose_y - y_min
        
        # Split face into left and right parts based on nose tip
        face_left = face_roi[:, :nose_x_rel]  # Left side of face
        face_right = face_roi[:, nose_x_rel:]  # Right side of face
        
        return face_left, face_right, (nose_x_rel, nose_y_rel), face_roi

def process_dataset(dataset_dir="dataset", output_dir="outputs/split_faces"):
    """
    Process all images in the dataset directory
    """
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    dataset_path = Path(dataset_dir)
    
    # Process each person's folder
    for person_folder in dataset_path.iterdir():
        if person_folder.is_dir():
            person_name = person_folder.name
            person_output_dir = Path(output_dir) / person_name
            
            # Create directories for left and right face parts
            left_dir = person_output_dir / "left"
            right_dir = person_output_dir / "right"
            full_dir = person_output_dir / "full_face"
            
            os.makedirs(left_dir, exist_ok=True)
            os.makedirs(right_dir, exist_ok=True)
            os.makedirs(full_dir, exist_ok=True)
            
            print(f"Processing {person_name}...")
            
            # Process each image in person's folder
            image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
            for image_file in person_folder.iterdir():
                if image_file.suffix.lower() in image_extensions:
                    face_left, face_right, nose_pos, full_face = detect_and_split_face(
                        str(image_file), str(person_output_dir)
                    )
                    
                    if face_left is not None and face_right is not None:
                        # Save the split faces
                        base_name = image_file.stem
                        
                        cv2.imwrite(str(left_dir / f"{base_name}_left.jpg"), face_left)
                        cv2.imwrite(str(right_dir / f"{base_name}_right.jpg"), face_right)
                        cv2.imwrite(str(full_dir / f"{base_name}_full.jpg"), full_face)
                        
                        print(f"  Processed: {image_file.name}")
                    else:
                        print(f"  Failed to process: {image_file.name}")

def visualize_split(image_path):
    """
    Visualize the face splitting process
    """
    face_left, face_right, nose_pos, full_face = detect_and_split_face(image_path, "outputs")
    
    if face_left is not None and face_right is not None:
        # Create visualization
        fig, axes = plt.subplots(1, 4, figsize=(16, 4))
        
        # Original image
        original = cv2.imread(image_path)
        original_rgb = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
        axes[0].imshow(original_rgb)
        axes[0].set_title('Original Image')
        axes[0].axis('off')
        
        # Full face
        full_face_rgb = cv2.cvtColor(full_face, cv2.COLOR_BGR2RGB)
        axes[1].imshow(full_face_rgb)
        axes[1].axvline(x=nose_pos[0], color='red', linestyle='--', linewidth=2)
        axes[1].set_title('Face with Nose Line')
        axes[1].axis('off')
        
        # Left face
        face_left_rgb = cv2.cvtColor(face_left, cv2.COLOR_BGR2RGB)
        axes[2].imshow(face_left_rgb)
        axes[2].set_title('Left Face')
        axes[2].axis('off')
        
        # Right face
        face_right_rgb = cv2.cvtColor(face_right, cv2.COLOR_BGR2RGB)
        axes[3].imshow(face_right_rgb)
        axes[3].set_title('Right Face')
        axes[3].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        return True
    else:
        print("Could not process the image")
        return False

# Example usage:
print("Face Detection and Splitting Tool")
print("=" * 40)

# Test with a single image first
test_image_path = "dataset/Arman/image_001.jpg"  # Adjust path as needed
if os.path.exists(test_image_path):
    print(f"Testing with: {test_image_path}")
    visualize_split(test_image_path)
else:
    print("Test image not found. Please adjust the path.")

# Process entire dataset
print("\nProcessing entire dataset...")
process_dataset()

print("\nProcessing complete! Check the 'outputs/split_faces' directory for results.")